In [3]:
import pybullet as p
import numpy as np
import pybullet_data
import time
from math import floor
import matplotlib.pyplot as plt
import gym
from gym import spaces

In [6]:
class Differential_Drive(gym.Env):
    metadata = {
        "render_modes" : None
    }
    def __init__(self):
        self.action_space = spaces.Discrete(4)

        self.observation_space = spaces.Tuple(
            (spaces.Box(low=-0.707,high = 0.707),spaces.Box(low=-0.707,high = 0.707),spaces.Box(low= 0, high = 10), spaces.Box(0,3),
            spaces.Box(-3.5,3.5),spaces.Box(0,7))
            )
        self.Kp = 6
        self.Ki = 0.3
        self.Kd = 0.3
        self.Kpw = 3 #1.5 smooth turns
        self.Kiw  =0
        self.Kdw = 0.3
        self.reference_velocity = 0.3048
        self.e_old = 0
        self.e_new = 0
        self.E_new = 0
        self.startPos = [-2,1,0]
        self.finalPos = [2,5,0]
        self.w_old = 0
        self.W_new = 0
        self.left_flag = 0
        self.right_flag = 1
        self.straight_flag = 2
        self.stop_flag = 3
        self.done = 0
        #distance = find_distance_origin(finalPos[0],finalPos[1])
        self.maxForce = 500 #this is a way of specifying the accelearation given the mass of the robot
        self.velocity_array = []
        self.angle_array = []
        self.reward = 0
        #self.info = 0 #to be decided later
        
    def reset(self):
        self.startPos = [-2,1,0]
        self.finalPos = [2,5,0]
        self.physicsClient = p.connect(p.GUI)
        p.setAdditionalSearchPath(pybullet_data.getDataPath())
        p.setGravity(0,0,-9.81)
        self.planeId = p.loadURDF("plane.urdf")
        self.mapId = p.loadURDF("./map3.urdf")
        self.error_plateId = p.loadURDF("./error_plate.urdf",[2,5,0])
        #p.changeDynamics(planeId,-1,lateralFriction = 0.2)
        self.startOrientation = p.getQuaternionFromEuler([0,0,0])
        print("this is the start orientation",self.startOrientation)
        self.robotId = p.loadURDF("4_wheel_differential_robot.urdf",self.startPos, self.startOrientation)
        
    def velocity_controller(self,error):
        e_old = self.e_old
        E_new = self.E_new
        e_dot = (error - e_old) #the Kd part of the constant covers the delta_t or the time step part too.
        E_new = E_new + error #this the summation of all the errors, and is used in the integtal part of u_t (del t is part of Ki)
        u = self.Kp*error + self.Kd*e_dot + self.Ki*E_new
        e_old = error
        return u #the final velocity  
    def find_distance(x1,y1,x2,y2) ->int: 
        distance = np.sqrt((x2-x1)**2 + (y2-y1)**2)
        return format(distance, f".{4}f")
    def angular_controller(self,error):
        w_old  = self.w_old
        W_new = self.W_new
        w_dot = (error - w_old) #the Kd part of the constant covers the delta_t or the time step part too.
        W_new = W_new + error #this the summation of all the errors, and is used in the integtal part of u_t (del t is part of Ki)
        w = self.Kpw*error + self.Kdw*w_dot + self.Kiw*W_new
        w_old = error
        return w #the final angular velocity 
    def reward_func(self,action,error_in_distance,beta,alpha):
        reward = 0
        angle_error = beta - alpha
        if angle_error > 0:
            #go left
            if action == 0:
                reward = reward + 10   
        elif angle_error < 0:
            # go right
            if action ==1:
                reward = reward + 10
        else:
            #go straight
            if action == 2:
                reward = reward + 10
        if error_in_distance < np.sqrt((self.startPos[0]-self.finalPos[0])**2 + (self.startPos[1]-self.finalPos[1])**2):
                reward = reward + (1/error_in_distance)*10
        if error_in_distance > np.sqrt((self.startPos[0]-self.finalPos[0])**2 + (self.startPos[1]-self.finalPos[1])**2):
            reward = reward - (error_in_distance)*10
        if error_in_distance < 0.1 : 
            reward = reward + 200
        print("reward: ",reward)
        return reward
    def step(self,action):
        finalPos = self.finalPos
        robotId = self.robotId
        maxForce = self.maxForce
        velocity_array = self.velocity_array
        reference_velocity = self.reference_velocity
        angle_array = self.angle_array
        pi = np.pi
        linear_velocity_in_x = p.getBaseVelocity(robotId)[0][0]
        linear_velocity_in_y = p.getBaseVelocity(robotId)[0][1]
        linear_velocity = np.sqrt(linear_velocity_in_x**2 + linear_velocity_in_y**2)
        velocity_array.append(linear_velocity)
        error_in_velocity = reference_velocity - linear_velocity
        error_in_distance_in_x = finalPos[0] - p.getBasePositionAndOrientation(robotId)[0][0]  
        error_in_distance_in_y = finalPos[1] - p.getBasePositionAndOrientation(robotId)[0][1]
        error_in_distance = np.sqrt(error_in_distance_in_x**2 + error_in_distance_in_y**2)
        print(error_in_distance,"m")
        target_velocity = self.velocity_controller(error_in_velocity)
        error_in_angle = np.arctan((finalPos[1]-p.getBasePositionAndOrientation(robotId)[0][1])/(finalPos[0]-p.getBasePositionAndOrientation(robotId)[0][0]))
        target_angle = self.angular_controller(error_in_angle)
        angle_array.append(target_angle)
        reward = 0
        value = p.getQuaternionFromEuler([0,0,np.arctan((finalPos[1]-p.getBasePositionAndOrientation(robotId)[0][1])/(finalPos[0]-p.getBasePositionAndOrientation(robotId)[0][0]))])
        if action == 0:
            #go left
                    info = 0
                    error_in_x = abs(finalPos[0] - p.getBasePositionAndOrientation(robotId)[0][0])
                    error_in_y = abs(finalPos[1] - p.getBasePositionAndOrientation(robotId)[0][1])
                    alpha = float(format(p.getBasePositionAndOrientation(robotId)[1][2],f".{3}f"))
                    beta = float(format(p.getQuaternionFromEuler([0,0,np.arctan((finalPos[1]-p.getBasePositionAndOrientation(robotId)[0][1])/(finalPos[0]-p.getBasePositionAndOrientation(robotId)[0][0]))])[2],f".{4}f"))
                    if beta < 0:
                        beta = float(format(p.getQuaternionFromEuler([0,0,pi - np.arctan((finalPos[1]-p.getBasePositionAndOrientation(robotId)[0][1])/(finalPos[0]-p.getBasePositionAndOrientation(robotId)[0][0]))])[2],f".{4}f"))           
                    error_in_x = abs(finalPos[0] - p.getBasePositionAndOrientation(robotId)[0][0])
                    error_in_y = abs(finalPos[1] - p.getBasePositionAndOrientation(robotId)[0][1])
                    error_in_distance = np.sqrt((finalPos[0] - p.getBasePositionAndOrientation(robotId)[0][0])**2 + (finalPos[1] - p.getBasePositionAndOrientation(robotId)[0][1])**2)
                    linear_velocity_in_x = p.getBaseVelocity(robotId)[0][0]
                    linear_velocity_in_y = p.getBaseVelocity(robotId)[0][1]
                    linear_velocity = np.sqrt(linear_velocity_in_x**2 + linear_velocity_in_y**2)
                    print("alpha",alpha)
                    print("beta",beta)
                    print("angle: ",p.getEulerFromQuaternion(p.getBasePositionAndOrientation(robotId)[1]))
                    print("error in x = ",error_in_x,"m")
                    print("error in y = ",error_in_y,"m")
                    print("taking left")
                    print("distance: ",error_in_distance,"m")
                    posx = p.getBasePositionAndOrientation(robotId)[0][0]
                    posy = p.getBasePositionAndOrientation(robotId)[0][1]
                    print("posx",p.getBasePositionAndOrientation(robotId)[0][0])
                    print("posy",p.getBasePositionAndOrientation(robotId)[0][1])
                    error_in_angle = np.arctan((finalPos[1]-p.getBasePositionAndOrientation(robotId)[0][1])/(finalPos[0]-p.getBasePositionAndOrientation(robotId)[0][0]))
                    #error_in_angle = reference_angle - np.arctan((finalPos[1]-p.getBasePositionAndOrientation(robotId)[0][1])/(finalPos[0]-p.getBasePositionAndOrientation(robotId)[0][0]))
                    target_velocity = self.velocity_controller(error_in_velocity)
                    target_angle = self.angular_controller(error_in_angle)
                    print("target_velocity of wheels: ",target_velocity)
                    print("target_angle: ",target_angle)
                    print("linear_velocity: ",linear_velocity)
                    p.setJointMotorControl2(bodyUniqueId=robotId,jointIndex=0,controlMode=p.VELOCITY_CONTROL,targetVelocity = -(target_velocity + 0.3048*0.5*target_angle)/0.04 ,force = maxForce)
                    p.setJointMotorControl2(bodyUniqueId=robotId,jointIndex=2,controlMode=p.VELOCITY_CONTROL,targetVelocity = -((linear_velocity - 0.3048*0.5*target_angle)/0.04) ,force = maxForce)
                    p.setJointMotorControl2(bodyUniqueId=robotId,jointIndex=1,controlMode=p.VELOCITY_CONTROL,targetVelocity = -(target_velocity + 0.3048*0.5*target_angle)/0.04 ,force = maxForce)
                    p.setJointMotorControl2(bodyUniqueId=robotId,jointIndex=3,controlMode=p.VELOCITY_CONTROL,targetVelocity = -((linear_velocity - 0.3048*0.5*target_angle)/0.04) ,force = maxForce)
                    p.stepSimulation()
                    time.sleep(1./50.)
                    print("\n")
                    reward = reward + self.reward_func(action,error_in_distance,beta,alpha)
                    #return alpha,beta,error_in_distance
                    
                    error_in_distance = np.sqrt((finalPos[0] - p.getBasePositionAndOrientation(robotId)[0][0])**2 + (finalPos[1] - p.getBasePositionAndOrientation(robotId)[0][1])**2)
                    linear_velocity_in_x = p.getBaseVelocity(robotId)[0][0]
                    linear_velocity_in_y = p.getBaseVelocity(robotId)[0][1]
                    linear_velocity = np.sqrt(linear_velocity_in_x**2 + linear_velocity_in_y**2)
                    alpha = float(format(p.getBasePositionAndOrientation(robotId)[1][2],f".{3}f"))
                    beta = float(format(p.getQuaternionFromEuler([0,0,np.arctan((finalPos[1]-p.getBasePositionAndOrientation(robotId)[0][1])/(finalPos[0]-p.getBasePositionAndOrientation(robotId)[0][0]))])[2],f".{4}f"))
                    if beta < 0:
                        beta = float(format(p.getQuaternionFromEuler([0,0,pi - np.arctan((finalPos[1]-p.getBasePositionAndOrientation(robotId)[0][1])/(finalPos[0]-p.getBasePositionAndOrientation(robotId)[0][0]))])[2],f".{4}f"))           
                    posx = p.getBasePositionAndOrientation(robotId)[0][0]
                    posy = p.getBasePositionAndOrientation(robotId)[0][1]
                    
                    
                    return [alpha,beta,error_in_distance,linear_velocity,posx,posy],reward,self.done,info
        elif action==1:
            #go right
                    info = 1
                    error_in_x = abs(finalPos[0] - p.getBasePositionAndOrientation(robotId)[0][0])
                    error_in_y = abs(finalPos[1] - p.getBasePositionAndOrientation(robotId)[0][1])
                    alpha = float(format(p.getBasePositionAndOrientation(robotId)[1][2],f".{3}f"))
                    beta = float(format(p.getQuaternionFromEuler([0,0,np.arctan((finalPos[1]-p.getBasePositionAndOrientation(robotId)[0][1])/(finalPos[0]-p.getBasePositionAndOrientation(robotId)[0][0]))])[2],f".{4}f"))
                    if beta < 0:
                        beta = float(format(p.getQuaternionFromEuler([0,0,pi - np.arctan((finalPos[1]-p.getBasePositionAndOrientation(robotId)[0][1])/(finalPos[0]-p.getBasePositionAndOrientation(robotId)[0][0]))])[2],f".{4}f"))
                    print("alpha",alpha)
                    print("beta",beta)
                    print("angle: ",p.getEulerFromQuaternion(p.getBasePositionAndOrientation(robotId)[1]))
                    
                    error_in_x = abs(finalPos[0] - p.getBasePositionAndOrientation(robotId)[0][0])
                    error_in_y = abs(finalPos[1] - p.getBasePositionAndOrientation(robotId)[0][1])
                    #error_in_angle = reference_angle - np.arctan((finalPos[1]-p.getBasePositionAndOrientation(robotId)[0][1])/(finalPos[0]-p.getBasePositionAndOrientation(robotId)[0][0])
                    error_in_angle = np.arctan((finalPos[1]-p.getBasePositionAndOrientation(robotId)[0][1])/(finalPos[0]-p.getBasePositionAndOrientation(robotId)[0][0]))
                    target_angle = self.angular_controller(error_in_angle)
                    error_in_distance = np.sqrt((finalPos[0] - p.getBasePositionAndOrientation(robotId)[0][0])**2 + (finalPos[1] - p.getBasePositionAndOrientation(robotId)[0][1])**2)
                    linear_velocity_in_x = p.getBaseVelocity(robotId)[0][0]
                    linear_velocity_in_y = p.getBaseVelocity(robotId)[0][1]
                    linear_velocity = np.sqrt(linear_velocity_in_x**2 + linear_velocity_in_y**2)
                    posx = p.getBasePositionAndOrientation(robotId)[0][0]
                    posy = p.getBasePositionAndOrientation(robotId)[0][1]
                    print("error in x = ",error_in_x,"m")
                    print("error in y = ",error_in_y,"m")
                    print("posx",p.getBasePositionAndOrientation(robotId)[0][0])
                    print("posy",p.getBasePositionAndOrientation(robotId)[0][1])
                    print("taking right")
                    target_velocity = self.velocity_controller(error_in_velocity)
                    print("distance: ",error_in_distance,"m")
                    print("target_velocity of wheels: ",target_velocity)
                    print("target_angle: ",target_angle)
                    print("linear_velocity: ",linear_velocity)
                    p.setJointMotorControl2(bodyUniqueId=robotId,jointIndex=0,controlMode=p.VELOCITY_CONTROL,targetVelocity = -((linear_velocity - 0.3048*0.5*(target_angle))/0.04) ,force = maxForce)
                    p.setJointMotorControl2(bodyUniqueId=robotId,jointIndex=2,controlMode=p.VELOCITY_CONTROL,targetVelocity = -((target_velocity + 0.3048*0.5*(target_angle))/0.04) ,force = maxForce)
                    p.setJointMotorControl2(bodyUniqueId=robotId,jointIndex=1,controlMode=p.VELOCITY_CONTROL,targetVelocity = -((linear_velocity - 0.3048*0.5*target_angle)/0.04) ,force = maxForce)
                    p.setJointMotorControl2(bodyUniqueId=robotId,jointIndex=3,controlMode=p.VELOCITY_CONTROL,targetVelocity = -((target_velocity + 0.3048*0.5*(target_angle))/0.04) ,force = maxForce)
                    
                    p.stepSimulation()
                    time.sleep(1./50.)
                    print("\n")
                    reward = reward + self.reward_func(action,error_in_distance,beta,alpha)
                    
                    error_in_distance = np.sqrt((finalPos[0] - p.getBasePositionAndOrientation(robotId)[0][0])**2 + (finalPos[1] - p.getBasePositionAndOrientation(robotId)[0][1])**2)
                    linear_velocity_in_x = p.getBaseVelocity(robotId)[0][0]
                    linear_velocity_in_y = p.getBaseVelocity(robotId)[0][1]
                    linear_velocity = np.sqrt(linear_velocity_in_x**2 + linear_velocity_in_y**2)
                    alpha = float(format(p.getBasePositionAndOrientation(robotId)[1][2],f".{3}f"))
                    beta = float(format(p.getQuaternionFromEuler([0,0,np.arctan((finalPos[1]-p.getBasePositionAndOrientation(robotId)[0][1])/(finalPos[0]-p.getBasePositionAndOrientation(robotId)[0][0]))])[2],f".{4}f"))
                    if beta < 0:
                        beta = float(format(p.getQuaternionFromEuler([0,0,pi - np.arctan((finalPos[1]-p.getBasePositionAndOrientation(robotId)[0][1])/(finalPos[0]-p.getBasePositionAndOrientation(robotId)[0][0]))])[2],f".{4}f"))           
                    posx = p.getBasePositionAndOrientation(robotId)[0][0]
                    posy = p.getBasePositionAndOrientation(robotId)[0][1]
                    

                    
                    #return alpha,beta,error_in_distance
                    return [alpha,beta,error_in_distance,linear_velocity,posx,posy],reward,self.done,info
        elif action==2:
            #go straight
                    info = 2
                    error_in_x = abs(finalPos[0] - p.getBasePositionAndOrientation(robotId)[0][0])
                    error_in_y = abs(finalPos[1] - p.getBasePositionAndOrientation(robotId)[0][1])
                    alpha = float(format(p.getBasePositionAndOrientation(robotId)[1][2],f".{3}f"))
                    beta = float(format(p.getQuaternionFromEuler([0,0,np.arctan((finalPos[1]-p.getBasePositionAndOrientation(robotId)[0][1])/(finalPos[0]-p.getBasePositionAndOrientation(robotId)[0][0]))])[2],f".{4}f"))
                    if beta < 0:
                        beta = float(format(p.getQuaternionFromEuler([0,0,pi - np.arctan((finalPos[1]-p.getBasePositionAndOrientation(robotId)[0][1])/(finalPos[0]-p.getBasePositionAndOrientation(robotId)[0][0]))])[2],f".{4}f"))
                    error_in_distance = np.sqrt((finalPos[0] - p.getBasePositionAndOrientation(robotId)[0][0])**2 + (finalPos[1] - p.getBasePositionAndOrientation(robotId)[0][1])**2)
                    print("alpha",alpha)
                    print("beta",beta)
                    print("angle: ",p.getEulerFromQuaternion(p.getBasePositionAndOrientation(robotId)[1]))
                    
                    error_in_x = abs(finalPos[0] - p.getBasePositionAndOrientation(robotId)[0][0])
                    error_in_y = abs(finalPos[1] - p.getBasePositionAndOrientation(robotId)[0][1])
                    
                    #error_in_angle = reference_angle - np.arctan((finalPos[1]-p.getBasePositionAndOrientation(robotId)[0][1])/(finalPos[0]-p.getBasePositionAndOrientation(robotId)[0][0]))
                    error_in_angle = np.arctan((finalPos[1]-p.getBasePositionAndOrientation(robotId)[0][1])/(finalPos[0]-p.getBasePositionAndOrientation(robotId)[0][0]))
      
                    target_angle = self.angular_controller(error_in_angle)
                    error_in_distance = np.sqrt((finalPos[0] - p.getBasePositionAndOrientation(robotId)[0][0])**2 + (finalPos[1] - p.getBasePositionAndOrientation(robotId)[0][1])**2)
                    linear_velocity_in_x = p.getBaseVelocity(robotId)[0][0]
                    linear_velocity_in_y = p.getBaseVelocity(robotId)[0][1]
                    linear_velocity = np.sqrt(linear_velocity_in_x**2 + linear_velocity_in_y**2)
                    posx = p.getBasePositionAndOrientation(robotId)[0][0]
                    posy = p.getBasePositionAndOrientation(robotId)[0][1]
                    print("error in x = ",error_in_x,"m")
                    print("error in y = ",error_in_y,"m")
                    print("posx",p.getBasePositionAndOrientation(robotId)[0][0])
                    print("posy",p.getBasePositionAndOrientation(robotId)[0][1])
                    print("taking straight")
                    target_velocity = self.velocity_controller(error_in_velocity)
                    print("distance: ",error_in_distance,"m")
                    print("target_velocity of wheels: ",target_velocity)
                    print("target_angle: ",target_angle)
                    print("linear_velocity: ",linear_velocity)
                    p.setJointMotorControl2(bodyUniqueId=robotId,jointIndex=0,controlMode=p.VELOCITY_CONTROL,targetVelocity = -abs(target_velocity)  ,force = maxForce)
                    p.setJointMotorControl2(bodyUniqueId=robotId,jointIndex=2,controlMode=p.VELOCITY_CONTROL,targetVelocity = -abs(target_velocity) ,force = maxForce)
                    p.setJointMotorControl2(bodyUniqueId=robotId,jointIndex=1,controlMode=p.VELOCITY_CONTROL,targetVelocity = -abs(target_velocity) ,force = maxForce)
                    p.setJointMotorControl2(bodyUniqueId=robotId,jointIndex=3,controlMode=p.VELOCITY_CONTROL,targetVelocity = -abs(target_velocity) ,force = maxForce)
                    
                    p.stepSimulation()
                    time.sleep(1./50.)
                    print("\n")
                    
                    reward = reward + self.reward_func(action,error_in_distance,beta,alpha)
                    #return alpha,beta,error_in_distance
                    
                    error_in_distance = np.sqrt((finalPos[0] - p.getBasePositionAndOrientation(robotId)[0][0])**2 + (finalPos[1] - p.getBasePositionAndOrientation(robotId)[0][1])**2)
                    linear_velocity_in_x = p.getBaseVelocity(robotId)[0][0]
                    linear_velocity_in_y = p.getBaseVelocity(robotId)[0][1]
                    linear_velocity = np.sqrt(linear_velocity_in_x**2 + linear_velocity_in_y**2)
                    alpha = float(format(p.getBasePositionAndOrientation(robotId)[1][2],f".{3}f"))
                    beta = float(format(p.getQuaternionFromEuler([0,0,np.arctan((finalPos[1]-p.getBasePositionAndOrientation(robotId)[0][1])/(finalPos[0]-p.getBasePositionAndOrientation(robotId)[0][0]))])[2],f".{4}f"))
                    if beta < 0:
                        beta = float(format(p.getQuaternionFromEuler([0,0,pi - np.arctan((finalPos[1]-p.getBasePositionAndOrientation(robotId)[0][1])/(finalPos[0]-p.getBasePositionAndOrientation(robotId)[0][0]))])[2],f".{4}f"))           
                    posx = p.getBasePositionAndOrientation(robotId)[0][0]
                    posy = p.getBasePositionAndOrientation(robotId)[0][1]
                    
                    
                    return [alpha,beta,error_in_distance,linear_velocity,posx,posy],reward, self.done, info
        else:
            #stops
            linear_velocity_in_x = p.getBaseVelocity(robotId)[0][0]
            linear_velocity_in_y = p.getBaseVelocity(robotId)[0][1]
            linear_velocity = np.sqrt(linear_velocity_in_x**2 + linear_velocity_in_y**2) 
            posx = p.getBasePositionAndOrientation(robotId)[0][0]
            posy = p.getBasePositionAndOrientation(robotId)[0][1]
            alpha = float(format(p.getBasePositionAndOrientation(robotId)[1][2],f".{3}f"))
            beta = float(format(p.getQuaternionFromEuler([0,0,np.arctan((finalPos[1]-p.getBasePositionAndOrientation(robotId)[0][1])/(finalPos[0]-p.getBasePositionAndOrientation(robotId)[0][0]))])[2],f".{4}f"))
            if beta < 0:
                    beta = float(format(p.getQuaternionFromEuler([0,0,pi - np.arctan((finalPos[1]-p.getBasePositionAndOrientation(robotId)[0][1])/(finalPos[0]-p.getBasePositionAndOrientation(robotId)[0][0]))])[2],f".{4}f"))
            reward = reward + self.reward_func(action,error_in_distance,beta,alpha)
            info = 3
            if (posx >= 3.3476 or posy >= 6.8476):
                print("STOPPING 2")
                p.setJointMotorControl2(bodyUniqueId=robotId,jointIndex=0,controlMode=p.VELOCITY_CONTROL,targetVelocity = 0 ,force = maxForce)
                p.setJointMotorControl2(bodyUniqueId=robotId,jointIndex=1,controlMode=p.VELOCITY_CONTROL,targetVelocity = 0 ,force = maxForce)
                p.setJointMotorControl2(bodyUniqueId=robotId,jointIndex=2,controlMode=p.VELOCITY_CONTROL,targetVelocity = 0 ,force = maxForce)
                p.setJointMotorControl2(bodyUniqueId=robotId,jointIndex=3,controlMode=p.VELOCITY_CONTROL,targetVelocity = 0 ,force = maxForce)
                p.stepSimulation()
                self.done = 1
                time.sleep(1./50.)
                
            #return alpha,beta,error_in_distance
                error_in_distance = np.sqrt((finalPos[0] - p.getBasePositionAndOrientation(robotId)[0][0])**2 + (finalPos[1] - p.getBasePositionAndOrientation(robotId)[0][1])**2)
                linear_velocity_in_x = p.getBaseVelocity(robotId)[0][0]
                linear_velocity_in_y = p.getBaseVelocity(robotId)[0][1]
                linear_velocity = np.sqrt(linear_velocity_in_x**2 + linear_velocity_in_y**2)
                alpha = float(format(p.getBasePositionAndOrientation(robotId)[1][2],f".{3}f"))
                beta = float(format(p.getQuaternionFromEuler([0,0,np.arctan((finalPos[1]-p.getBasePositionAndOrientation(robotId)[0][1])/(finalPos[0]-p.getBasePositionAndOrientation(robotId)[0][0]))])[2],f".{4}f"))
                if beta < 0:
                    beta = float(format(p.getQuaternionFromEuler([0,0,pi - np.arctan((finalPos[1]-p.getBasePositionAndOrientation(robotId)[0][1])/(finalPos[0]-p.getBasePositionAndOrientation(robotId)[0][0]))])[2],f".{4}f"))           
                posx = p.getBasePositionAndOrientation(robotId)[0][0]
                posy = p.getBasePositionAndOrientation(robotId)[0][1]

            
                return [alpha,beta,error_in_distance,linear_velocity,posx,posy],reward,self.done,info
            
            if(error_in_distance < 0.1):
                print("STOPPING 1")
                p.setJointMotorControl2(bodyUniqueId=robotId,jointIndex=0,controlMode=p.VELOCITY_CONTROL,targetVelocity = 0 ,force = maxForce)
                p.setJointMotorControl2(bodyUniqueId=robotId,jointIndex=1,controlMode=p.VELOCITY_CONTROL,targetVelocity = 0 ,force = maxForce)
                p.setJointMotorControl2(bodyUniqueId=robotId,jointIndex=2,controlMode=p.VELOCITY_CONTROL,targetVelocity = 0 ,force = maxForce)
                p.setJointMotorControl2(bodyUniqueId=robotId,jointIndex=3,controlMode=p.VELOCITY_CONTROL,targetVelocity = 0 ,force = maxForce)
                p.stepSimulation()
                self.done = 1
                time.sleep(1./50.)
                
                       
            #return alpha,beta,error_in_distance
            
            error_in_distance = np.sqrt((finalPos[0] - p.getBasePositionAndOrientation(robotId)[0][0])**2 + (finalPos[1] - p.getBasePositionAndOrientation(robotId)[0][1])**2)
            linear_velocity_in_x = p.getBaseVelocity(robotId)[0][0]
            linear_velocity_in_y = p.getBaseVelocity(robotId)[0][1]
            linear_velocity = np.sqrt(linear_velocity_in_x**2 + linear_velocity_in_y**2)
            alpha = float(format(p.getBasePositionAndOrientation(robotId)[1][2],f".{3}f"))
            beta = float(format(p.getQuaternionFromEuler([0,0,np.arctan((finalPos[1]-p.getBasePositionAndOrientation(robotId)[0][1])/(finalPos[0]-p.getBasePositionAndOrientation(robotId)[0][0]))])[2],f".{4}f"))
            if beta < 0:
                beta = float(format(p.getQuaternionFromEuler([0,0,pi - np.arctan((finalPos[1]-p.getBasePositionAndOrientation(robotId)[0][1])/(finalPos[0]-p.getBasePositionAndOrientation(robotId)[0][0]))])[2],f".{4}f"))           
            posx = p.getBasePositionAndOrientation(robotId)[0][0]
            posy = p.getBasePositionAndOrientation(robotId)[0][1]
            return [alpha,beta,error_in_distance,linear_velocity,posx,posy],reward,self.done,info
    def render(self):
        p.stepSimulation()
        time.sleep(1./50.)
        pass
    def close(self):
        p.disconnect()

In [8]:

env = Differential_Drive()
action = env.action_space.sample() #taking sample actions
episode_number = 10
for i in range(episode_number):
    state = env.reset()
    done = False
    total_reward = 0
    while not done:
        action = env.action_space.sample()
        #env.render()
        
        [alpha,beta,error_in_distance,linear_velocity,posx,posy],reward,done,info  = env.step(action)
        total_reward=total_reward+reward
        for i in range(1000):
            [alpha,beta,error_in_distance,linear_velocity,posx,posy],reward,done,info = env.step(1)
            total_reward=total_reward+reward
            print(info)
        
        print("reward: ",reward)
        print("total_reward: ",total_reward)
        env.close()
        done = True
        
        

this is the start orientation (0.0, 0.0, 0.0, 1.0)
5.656854249492381 m
reward:  0
5.656854249492381 m
alpha 0.0
beta 0.3827
angle:  (0.0, -0.0, 0.0)
error in x =  4.0 m
error in y =  4.0 m
posx -2.0
posy 1.0
taking right
distance:  5.656854249492381 m
target_velocity of wheels:  2.01168
target_angle:  2.5918139392115793
linear_velocity:  0.0


reward:  0
1
5.656545548400203 m
alpha -0.001
beta 0.3827
angle:  (6.80649082905283e-09, 1.3491661159144723e-09, -0.0010357419894261712)
error in x =  3.9995091087569032 m
error in y =  4.0000543033934814 m
posx -1.9995091087569032
posy 0.9999456966065184
taking right
distance:  5.656545548400203 m
target_velocity of wheels:  1.2293650729639585
target_angle:  2.592038844272672
linear_velocity:  0.11853256470243055


reward:  1.7678634273223917
1
5.656239225803593 m
alpha -0.001
beta 0.3827
angle:  (-6.913463560896358e-08, 2.201044589375551e-05, -0.0020712208518349562)
error in x =  3.9990215537295564 m
error in y =  4.0001085975665305 m
posx -1.9

error: Not connected to physics server.

In [ ]:
p.disconnect()